# LOAD DATASET

In [1]:
import pandas as pd
import numpy as np


concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()
 


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# PREPROCESSING

check for null values

In [2]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [3]:

concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
predictors.head()
 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [4]:

target.head()
 


0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

normalising the tarining data 

In [5]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [6]:
 
n_cols = predictors.shape[1] # number of predictors
n_cols




8

# MODEL BUILDING

In [7]:

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

# build the model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
 


# TRAINING

In [8]:
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/100
23/23 [==============================] - 1s 1ms/step - loss: 43071.7734
Epoch 2/100
23/23 [==============================] - 0s 1ms/step - loss: 22129.4355
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 12767.4775
Epoch 4/100
23/23 [==============================] - 0s 1ms/step - loss: 8250.1562
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 5729.1650
Epoch 6/100
23/23 [==============================] - 0s 1ms/step - loss: 4330.1367
Epoch 7/100
23/23 [==============================] - 0s 1ms/step - loss: 3475.7722
Epoch 8/100
23/23 [==============================] - 0s 1ms/step - loss: 2922.4392
Epoch 9/100
23/23 [==============================] - 0s 1ms/step - loss: 2551.6331
Epoch 10/100
23/23 [==============================] - 0s 1ms/step - loss: 2306.8701
Epoch 11/100
23/23 [==============================] - 0s 4ms/step - loss: 2139.2764
Epoch 12/100
23/23 [==============================] - 0s 1ms/step - loss: 2017.929

In [9]:
from sklearn.metrics import mean_squared_error
y_pred=model.predict(X_test)
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)
 


10/10 [==============================] - 0s 889us/step
1384.722943691772 0.0


# EVALUATION

In [11]:

total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with  normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 256.72064208984375
10/10 [==============================] - 0s 2ms/step
MSE 2: 262.5904846191406
10/10 [==============================] - 0s 1ms/step
MSE 3: 266.16363525390625
10/10 [==============================] - 0s 2ms/step
MSE 4: 299.8914489746094
10/10 [==============================] - 0s 1ms/step
MSE 5: 297.4675598144531
10/10 [==============================] - 0s 2ms/step
MSE 6: 244.58694458007812
10/10 [==============================] - 0s 1ms/step
MSE 7: 327.98297119140625
10/10 [==============================] - 0s 1ms/step
MSE 8: 272.7510681152344
10/10 [==============================] - 0s 1ms/step
MSE 9: 273.71380615234375
10/10 [==============================] - 0s 1ms/step
MSE 10: 254.31134033203125
10/10 [==============================] - 0s 1ms/step
MSE 11: 272.375244140625
10/10 [==============================] - 0s 1ms/step
MSE 12: 258.2835388183594
10/10 [==============================] - 0s 1ms/step
MSE 13: 252.4446258544922
10/10 [=======================

as we can see above the mean of MSE increased but SD decreased as compared to part B